In [78]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [79]:
IMAGE_SIZE = 512
BATCH_SIZE = 4
NUM_CLASSES = 20
DATA_DIR = "D:\\Study\\Segmentation\\instance-level-human-parsing\\instance-level_human_parsing\\instance-level_human_parsing\\Training"
NUM_TRAIN_IMAGES = 1000
NUM_VAL_IMAGES = 50

In [80]:
train_images = sorted(glob(os.path.join(DATA_DIR, 'Images/*')))[:NUM_TRAIN_IMAGES]#1000
train_masks = sorted(glob(os.path.join(DATA_DIR, 'Category_ids/*')))[:NUM_TRAIN_IMAGES]#1000

val_images = sorted(glob(os.path.join(DATA_DIR, 'Images/*')))[NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES]#1000
val_masks = sorted(glob(os.path.join(DATA_DIR, 'Category_ids/*')))[NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES]#1000

In [81]:
def read_image(image_path, mask = False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels = 1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images = image, size = [IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels = 3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images = image, size = [IMAGE_SIZE, IMAGE_SIZE])
        image = image / 127.5 - 1 # Normalization -1 ~ 1
    
    return image

In [82]:
def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask = True)
    return image, mask

In [83]:
def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [84]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset : ", train_dataset)
print("Val Dataset : ", val_dataset)

Train Dataset :  <BatchDataset shapes: ((4, 512, 512, 3), (4, 512, 512, 1)), types: (tf.float32, tf.float32)>
Val Dataset :  <BatchDataset shapes: ((4, 512, 512, 3), (4, 512, 512, 1)), types: (tf.float32, tf.float32)>


In [85]:
def convolution_block(block_input, num_filters=256, kernel_size = 3,
                      dilation_rate = 1, padding = 'same', use_bias = False):
    x = layers.Conv2D(num_filters, kernel_size = kernel_size,
                      dilation_rate=dilation_rate, padding = 'same',
                      use_bias=use_bias, kernel_initializer = keras.initializers.HeNormal())(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)

In [86]:
def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size = (dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size = 1, use_bias = True)
    out_pool = layers.UpSampling2D(size = (dims[-3] // x.shape[1], dims[-2] // x.shape[2]),
                                   interpolation='bilinear')(x)
    
    out_1 = convolution_block(dspp_input, kernel_size = 1, dilation_rate = 1)
    out_6 = convolution_block(dspp_input, kernel_size = 3, dilation_rate = 6)
    out_12 = convolution_block(dspp_input, kernel_size = 3, dilation_rate = 12)
    out_18 = convolution_block(dspp_input, kernel_size = 3, dilation_rate = 18)
    
    x = layers.Concatenate(axis = -1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size = 1)
    return output

In [125]:
def DeeplabV3(image_size, num_classes):
    model_input = keras.Input(shape = (image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(weights='imagenet', include_top = False, input_tensor = model_input)
    
    x = resnet50.get_layer('conv4_block6_2_relu').output
    x = DilatedSpatialPyramidPooling(x)
    
    input_a = layers.UpSampling2D(size = (image_size // 4 // x.shape[1],
                                          image_size // 4 // x.shape[2]),
                                  interpolation='bilinear')(x)
    
    input_b = resnet50.get_layer('conv2_block3_2_relu').output
    input_b = convolution_block(input_b, num_filters = 48, kernel_size = 1)
    
    x = layers.Concatenate(axis = -1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(size = (image_size // x.shape[1],
                                    image_size // x.shape[2]),
                            interpolation='bilinear')(x)
    model_output = layers.Conv2D(num_classes , kernel_size = (1, 1), padding = 'same')(x)
    return keras.Model(inputs = model_input, outputs = model_output)

model = DeeplabV3(image_size = IMAGE_SIZE, num_classes = NUM_CLASSES)
model.summary()

Model: "model_40"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_42 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_42[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [126]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001), loss = loss, metrics = ['accuracy'])

history = model.fit(train_dataset, validation_data = val_dataset, epochs = 20)



Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[4,256,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model_40/conv2d_409/Conv2D/Conv2DBackpropInput
 (defined at C:\ProgramData\Anaconda3\envs\segmentation\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:464)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_220450]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/model_40/conv2d_409/Conv2D/Conv2DBackpropInput:
In[0] gradient_tape/model_40/conv2d_409/Conv2D/ShapeN:	
In[1] model_40/conv2d_409/Conv2D/ReadVariableOp (defined at C:\ProgramData\Anaconda3\envs\segmentation\lib\site-packages\keras\layers\convolutional.py:231)	
In[2] gradient_tape/sparse_categorica